In [1]:
!nvidia-smi

Mon Apr  8 14:11:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|


|   0  NVIDIA GeForce RTX 4090        On  | 00000000:C2:00.0 Off |                  Off |
| 30%   27C    P8              31W / 450W |      3MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|  No running processes found                                                           |
+---------

In [10]:
from src.segmentation.framework_handlers import yolo_handler
yolov8_handler.validate_version_and_gpu()

Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
Setup complete ✅ (32 CPUs, 251.7 GB RAM, 31.8/332.3 GB disk)


### Prepare Dataset
In this section, the dataset along with YOLO annotations is processed to conform to the input structure required by YOLO. The YOLO format necessitates organizing the dataset into two main directories, `images` and `labels`. Each of these directories is further divided into two subdirectories, `train` and `val`, for training and validation datasets respectively. This structure facilitates YOLO's training and validation processes.

In [9]:
# prepare data to fine tune yolo v8
from src.segmentation.framework_handlers.yolo_handler import prepare_data_for_yolo
annotations_folder_name = "etaylor_cannabis_patches_test"
dataset_version = "v0.1"
saving_yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8"
yaml_file_path = prepare_data_for_yolo(annotations_folder_name, dataset_version, saving_yaml_path, train_percentage=0.8)

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/export_coco-instance_etaylor_cannabis_patches_test_v0.1.json:  53%|█████▎    | 47/89 [00:00<00:00, 465.70it/s]

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/export_coco-instance_etaylor_cannabis_patches_test_v0.1.json: 100%|██████████| 89/89 [00:00<00:00, 479.47it/s]

COCO data converted successfully.
Results saved to /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo


The folder /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo has been deleted.


### Ultralytics Settings

In here we can modify the settings of the yolo runs. 

In [3]:
from ultralytics import settings

runs_dir = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs"
# Update a setting
settings.update({'runs_dir': runs_dir})

# View all settings
print(settings)

{'settings_version': '0.0.4', 'datasets_dir': '/sise/home/etaylor/datasets', 'weights_dir': '/home/etaylor/weights', 'runs_dir': '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs', 'uuid': '958bdde9742b290d62523897603d59c1039672389661ad1a173c1e862ebe77c6', 'sync': True, 'api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': False, 'wandb': True}


### Fine tune YOLOv8 Model on my dataset

In [7]:
# in case dataset already converted from coco to yolo format use manual path
yaml_file_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/etaylor_cannabis_patches_test_v0.1_data.yaml"


In [8]:
from ultralytics import YOLO

model_checkpoint_path = '/home/etaylor/code_projects/thesis/checkpoints/yolo/yolov8n.pt'
model = YOLO(model_checkpoint_path)

# regular train of the model
results = model.train(data=yaml_file_path, epochs=300, imgsz=512)

Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
engine/trainer: task=detect, mode=train, model=/home/etaylor/code_projects/thesis/checkpoints/yolo/yolov8n.pt, data=/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/etaylor_cannabis_patches_test_v0.1_data.yaml, epochs=300, time=None, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fal

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo_split/labels/train.cache... 73 images, 70 backgrounds, 0 corrupt: 100%|██████████| 143/143 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/yolo8/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo_split/labels/val.cache... 16 images, 20 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/yolo8/lib/python3.10/site-packages/torch/uti

Plotting labels to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/300      1.59G      2.075      4.343      1.435         82        512: 100%|██████████| 9/9 [00:05<00:00,  1.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.72it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    131/300       1.5G     0.9178     0.7356     0.9083        112        512: 100%|██████████| 9/9 [00:00<00:00, 13.88it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 14.85it/s]


                   all         36        167      0.538      0.502      0.552       0.38
Stopping training early as no improvement observed in last 50 epochs. Best results observed at epoch 81, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

131 epochs completed in 0.058 hours.
Optimizer stripped from /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs/detect/train2/weights/best.pt, 6.2MB

Validating /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00, 10.71it/s]


                   all         36        167      0.474      0.709      0.648       0.45
              trichome         36         47      0.437      0.766      0.688      0.474
                 clear         36        106      0.667      0.575      0.688      0.497
                cloudy         36         14      0.319      0.786      0.567       0.38
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs/detect/train2


In [9]:
valid_results = model.val()
print(valid_results)

Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)


Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo_split/labels/val.cache... 16 images, 20 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]
/home/etaylor/.conda/envs/yolo8/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.06it/s]


                   all         36        167      0.476      0.709      0.647      0.449
              trichome         36         47      0.438      0.766      0.687      0.473
                 clear         36        106       0.67      0.575      0.688      0.493
                cloudy         36         14      0.319      0.786      0.567       0.38
Speed: 0.1ms preprocess, 64.4ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs/detect/train22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc1724d45e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.00700

In [ ]:
from IPython.display import Image as show_image
show_image(filename="/sise/home/etaylor/runs/segment/train22/val_batch0_pred.jpg")